In [2]:
import pennylane as qml
from pennylane import numpy as np

import matplotlib.pyplot as plt


In [3]:
dev = qml.device("default.qubit", wires=6)
# wires 0,1,2 are group B (input training data)
# wires 3,4,5 are group A (store past data)

# initialize a, J values (stays same throughout trainig)
a = []
J = np.zeros((6, 6))
for i in range(6):
  a.append( np.random.uniform(-1.0, 1.0) )
  for j in range(i):
    J[i][j] = np.random.uniform(-1.0, 1.0)

# calculate Hint
Hint = 0
for j in range(6):
    Hint = Hint + a[j]*qml.PauliX(wires=j)
    for k in range(j):
        Hint = Hint + J[j][k] * qml.PauliZ(wires=j) * qml.PauliZ(wires=k)

# some fixed values (as specified in paper)
D = 3
tau = 0.2

# Parameters
alpha_array = np.zeros((D, 6))
beta_array  = np.zeros((D, 6))
gamma_array = np.zeros((D, 6))
c = 1

def encoding(x):
    qml.RY(np.arccos(x), wires=0)
    qml.RY(np.arccos(x), wires=1)
    qml.RY(np.arccos(x), wires=2)

def U1(alpha, beta, gamma, wire):
     qml.RX(alpha, wires=wire)
     qml.RZ(beta, wires=wire)
     qml.RX(gamma, wires=wire)

def evolution():
    for i in range (D): # D = 3
        # U1 rotations
        for j in range(6):
            U1(alpha_array[i][j], beta_array[i][j], gamma_array[i][j], j)

        np.exp(-1.0*j * Hint * tau) # e^(-i Hint tau)

@qml.qnode(dev)
def pqc(x):
    encoding(x)
    evolution()
    
    return [qml.expval(qml.PauliZ(i)) for i in range(3)]

def timestep_output(x):
    result = pqc(x)
    print("Output of pqc: ", result)
    
    y = c * np.average(result)
    print("Output of each timestep: ", y)

    return y

timestep_output(0.7, 1)

# print(qml.draw_mpl(pqc)(0.5, 1))
# print("blahhhhhhhhh", np.arccos(1))


### random code ideas, unused

# expval1 = qml.expval(qml.PauliZ(0))
# expval2 = qml.expval(qml.PauliZ(1))
# expval3 = qml.expval(qml.PauliZ(2))

# avg = (expval1 + expval2 + expval3) / 3.0

# print([expval1.eigvals(), expval2.eigvals(), expval3.eigvals()])

# return c * avg
# return expval1, expval2, expval3

ValueError: Cannot multiply Hamiltonian by <class 'pennylane.ops.qubit.non_parametric_ops.PauliZ'>

In [4]:
def generate_input(t_in):
    t=8/199*t_in
    if t>=0 and t<=1:
        return -t+0.5
    elif t>=1 and t<=2:
        return t-1.5
    elif t>=2 and t<=3:
        return -t+2.5
    else:
        return t-3.5